https://www.learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/

In [ ]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

In [ ]:
# from torchvision import models
import torch
import torch
import torchvision #provides CV functionality and Datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
device = 'cuda'#changes CPU to run on GPU

hide_toggle()

TIME TO PRINT THE WEIGHTS OF A CNN IN IMAGE FORM

for p in resnet101.parameters():
    print(p)
    
for name, param in resnet101.state_dict().items():
    print(name)
    print(param)
    

for name, param in resnet101.state_dict().items():
    if 'conv' in name:
        print(name)
        

for name, layer in resnet101.state_dict().items():
    print(layer)

hide_toggle()

In [ ]:
import torchvision.models as models
resnet101 = models.resnet101(pretrained=True)

from torchvision import transforms
transform = transforms.Compose([            #[1]
    transforms.Resize(256),                    #[2]
    transforms.CenterCrop(224),                #[3]
    transforms.ToTensor(),                     #[4]
    transforms.Normalize(                      #[5]
    mean=[0.485, 0.456, 0.406],                #[6]
    std=[0.229, 0.224, 0.225]                  #[7]
 )])


from PIL import Image
img = Image.open("dog.jpg")
img_t=transform(img)
batch_t=torch.unsqueeze(img_t,0)

hide_toggle()

In [ ]:
resnet101


visulization using activation maximization
https://towardsdatascience.com/how-to-visualize-convolutional-features-in-40-lines-of-code-70b7d87b0030 
visulizing using hooks
https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/

# First Try:-

Just managed to extract features from the end of each bottlneck layer in the resnet101

### Dog input

In [ ]:
hide_toggle(for_next=True) #The hook for the resnet101 for the dog.jpg

In [ ]:
#first the sequntial layer is printed in its entirety then the bottleneck layers inside that sequential layer are printed
#thus the entire resnet101 is printed twice

net = resnet101
visualisation = {}

def hook_fn(m, i, o):
    visualisation[m] = o.clone().detach()

def get_all_layers(net):
    for name, layer in net._modules.items():
        print(layer)
        #If it is a sequential, don't register a hook on it
        # but recursively register hook on all it's module children
        if isinstance(layer, nn.Sequential):
            get_all_layers(layer)
        else:
            # it's a non sequential. Register a hook
            layer.register_forward_hook(hook_fn)


get_all_layers(net)

  
resnet101.eval()
out = resnet101(batch_t)
print(out.shape)

#printing the results of the input dog.jpg
'''with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
    
_, index = torch.max(out, 1)

percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

print(classes[index[0]],percentage[index[0]].item())'''


# Just to check whether we got all layers
#visualisation.keys()    #output includes sequential layers



In [ ]:
#printing all the layer information
#U will notice i didn't access any of the convo layers inside the bottlenecks
num = 1
for layer in visualisation.keys():
    print(str(num)+" "+str(layer)+" ")
    num+=1

In [ ]:
visualisation.values()

### Random Input

In [ ]:
#Generating a random image(random numpy array 3 color channels)
'''sz = 256
img = np.uint8(np.random.uniform(150, 180, (256, 256, 3)))/25
'''

#creating a pytorch variable
#first transform the img into a tensor then define Variable
img_t = 150*torch.randn(1,3,256,256)    #tensor
img = np.uint8(img_t.reshape(256,256,3))
plt.imshow(img)
#print(img_t)
img_var = Variable(img_t, requires_grad=True)#wrap the tensor in pytorch variable

hide_toggle()

In [ ]:
get_all_layers(net)
out = resnet101(img_var)
print(out.shape)

In [ ]:
visualisation.values()

# Second try:-


In [ ]:
#there are a total of 104 convolutional layers in the resnet101 structure 4 of which are part of sequential layers inside 
#the bottleneck layers and are there for downsampleing the image. The following layers will be used for extracting features
num=1
for name, layer in net.named_modules():
    if 'conv' in name:
        print(str(num)+" "+str(name))
        print(layer)
        num+=1

In [ ]:
for name,layer in net.named_modules():
    print(name)

In [ ]:
#this namelist will be used to name the images later on
n=0
namelist=[]
for name, layer in net.named_modules():
    if 'Bottleneck' not in str(layer):
        print(name)
        namelist.append(name)
        print(n)
        n+=1

### RANDOM

In [ ]:
#Registering hooks in each of the layers of the resnet
'''the way it registers is a bit weird 
First is hooks every individual layer inside a bottleneck then the bottleneck itself
after that it hooks the sequential layer as a whole after hooking all the bottleneck layers
finally the entire resnet is hooked'''

visualisation = {}

def hook_fn(m, i, o):
    visualisation[m] = o.clone().detach()

def get_features(net):
    for name, layer in net.named_modules():
        layer.register_forward_hook(hook_fn)

        
get_features(resnet101)
out = resnet101(img_var)
print(out.shape)

In [ ]:
for i in visualisation.keys():
    if 'Bottleneck'not in str(i):
        print('_________________')
        print(i)

In [ ]:
for i in visualisation.values():
    print(i)

In [ ]:
pos=0
for name in visualisation.keys():
    if 'Bottleneck' not in str(name):
        num=0
        for values in visualisation.values():
            if num==pos:
                print(str(name))
                print(namelist[pos])
                print(pos)
                pos+=1
                break
            num+=1
            

In [ ]:
pos=0
size = 256,256
for name in visualisation.keys():
    if 'Bottleneck' not in str(name):
        num=0
        for values in visualisation.values():
            if num==pos:
                print(values.size())
                print(pos)
                for filters in range(10):
                    if values.size(2)!=1:
                        img = np.asarray(values[0,filters].view(values.size(2),values.size(2)))
                        plt.imsave(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg", img)
                        im = Image.open(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg")
                        im_resized = im.resize(size)
                        im_resized.save(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg", "PNG")
                    else: 
                        break
                pos+=1
                break
            num+=1
            

### DOG

In [ ]:
resnet101.eval()
out = resnet101(batch_t)
print(out.shape)

In [ ]:
pos=0
size = 256,256
for name in visualisation.keys():
    if 'Bottleneck' not in str(name):
        num=0
        for values in visualisation.values():
            if num==pos:
                print(values.size())
                print(pos)
                for filters in range(10):
                    if values.size(2)!=1:
                        img = np.asarray(values[0,filters].view(values.size(2),values.size(2)))
                        plt.imsave(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg", img)
                        im = Image.open(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg")
                        im_resized = im.resize(size)
                        im_resized.save(str(pos)+'_'+str(namelist[pos])+"_filter_"+str(filters)+"_Tensor_size_"+str(values.size())+".jpg", "PNG")
                    else: 
                        break
                pos+=1
                break
            num+=1
            